In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm

In [2]:
operations = {
    "orf": "wellpos_cc_var_mad_outlier_featselect_sphering_harmony",
    "crispr": "wellpos_var_mad_int_featselect_harmony_PCA_corrected"
}

variable = "homologue_existence"
col_name = f"Metadata_{variable}"
col_name_in_metadata = "Metadata_homologue"
output_df = pd.DataFrame()

In [3]:
# Read phenotypic activity

orf_phenotypic_activity_df = pd.read_csv(
    f"../03.retrieve-annotations/output/phenotypic-activity-{operations['orf']}.csv.gz"
)[["Metadata_JCP2022", "mean_average_precision", "below_corrected_p"]]

crispr_phenotypic_activity_df = pd.read_csv(
    f"../03.retrieve-annotations/output/phenotypic-activity-{operations['crispr']}.csv.gz"
)[["Metadata_JCP2022", "mean_average_precision", "below_corrected_p"]]

In [4]:
# Read metadata

orf_metadata_df = (
    pd.read_csv(
        "../00.download-and-process-annotations/output/orf_metadata.tsv.gz", sep="\t"
    )[["Metadata_JCP2022", col_name_in_metadata]]
    .assign(col=lambda x: x[col_name_in_metadata])
    .rename(columns={"col": col_name})
)

crispr_metadata_df = (
    pd.read_csv(
        "../00.download-and-process-annotations/output/crispr_metadata.tsv.gz", sep="\t"
    )[["Metadata_JCP2022", col_name_in_metadata]]
    .assign(col=lambda x: x[col_name_in_metadata])
    .rename(columns={"col": col_name})
)

In [5]:
orf_df = orf_phenotypic_activity_df.merge(
    orf_metadata_df, on="Metadata_JCP2022", how="inner"
).dropna(subset=[col_name])
crispr_df = crispr_phenotypic_activity_df.merge(
    crispr_metadata_df, on="Metadata_JCP2022", how="inner"
).dropna(subset=[col_name])

Fisher's exact test ORF

In [6]:
table = sm.stats.Table.from_data(orf_df[["below_corrected_p", col_name]])
print(
    pd.DataFrame(
        table.table,
        columns=[f"{variable}_False", f"{variable}_True"],
        index=["phenotype_False", "phenotype_True"],
    ).to_markdown()
)

odds_ratio, pvalue = stats.fisher_exact(table.table)
print(f"odds ratio: {odds_ratio}, p-value: {pvalue}")

output_df = pd.concat(
    [
        output_df,
        pd.DataFrame(
            {
                "modality": ["ORF"],
                f"{variable}_False_phenotype_False": int(table.table[0, 0]),
                f"{variable}_False_phenotype_True": int(table.table[0, 1]),
                f"{variable}_True_phenotype_False": int(table.table[1, 0]),
                f"{variable}_True_phenotype_True": int(table.table[1, 1]),
                "odds_ratio": odds_ratio,
                "pvalue": pvalue,
            },
            index=[0],
        ),
    ], 
    ignore_index=True
)

|                 |   homologue_existence_False |   homologue_existence_True |
|:----------------|----------------------------:|---------------------------:|
| phenotype_False |                         150 |                       5036 |
| phenotype_True  |                         200 |                       6896 |
odds ratio: 1.0270055599682288, p-value: 0.8262674430677552


There is no correlation between homologue existence and phenotypic activity

Fisher's exact test CRISPR

In [7]:
table = sm.stats.Table.from_data(crispr_df[["below_corrected_p", col_name]])
print(
    pd.DataFrame(
        table.table,
        columns=[f"{variable}_False", f"{variable}_True"],
        index=["phenotype_False", "phenotype_True"],
    ).to_markdown()
)

odds_ratio, pvalue = stats.fisher_exact(table.table)
print(f"odds ratio: {odds_ratio}, p-value: {pvalue}")

output_df = pd.concat(
    [
        output_df,
        pd.DataFrame(
            {
                "modality": ["CRISPR"],
                f"{variable}_False_phenotype_False": int(table.table[0, 0]),
                f"{variable}_False_phenotype_True": int(table.table[0, 1]),
                f"{variable}_True_phenotype_False": int(table.table[1, 0]),
                f"{variable}_True_phenotype_True": int(table.table[1, 1]),
                "odds_ratio": odds_ratio,
                "pvalue": pvalue,
            },
            index=[0],
        ),
    ], 
    ignore_index=True
)


|                 |   homologue_existence_False |   homologue_existence_True |
|:----------------|----------------------------:|---------------------------:|
| phenotype_False |                          26 |                       2206 |
| phenotype_True  |                          43 |                       5426 |
odds ratio: 1.487233549094436, p-value: 0.11134113749119579


There is no correlation between homologue existence and phenotypic activity.

In [8]:
output_df.to_csv(f"output/{variable}.csv", index=False)